# Overview
- This notebook allows a user to explore evaluations of tested prompts.

In [29]:
import pandas as pd
import os
from glob import glob
from base import DATA_PATH

EVAL_DIR = '../data/evaluations'
files = glob(os.path.join(EVAL_DIR, '*.parquet'))
dfs = [pd.read_parquet(file) for file in files]
merged = pd.concat(dfs, ignore_index=True)

print(merged.shape)
cols = ['example_no','prompt_version','llm_bleu_score','llm_rouge_score','llm_f1']
example_1 = merged[merged.example_no=='example']
# example_2 = merged[merged.example_no=='example-1']
example_1[cols].sort_values(by='llm_bleu_score')

(10, 24)


,example_no,prompt_version,llm_bleu_score,llm_rouge_score,llm_f1
4,example,4,0.306604,"{'rouge-1': {'f': 0.591304342826465, 'p': 0.58...",0.75
5,example,5,0.317677,"{'rouge-1': {'f': 0.5614035037719299, 'p': 0.5...",0.75
8,example,3,0.364264,"{'rouge-1': {'f': 0.5840707914605686, 'p': 0.5...",0.75
9,example,2,0.389698,"{'rouge-1': {'f': 0.5794392473578479, 'p': 0.6...",0.75
3,example,6,0.432629,"{'rouge-1': {'f': 0.6140350827192983, 'p': 0.6...",0.75
1,example,7,0.449965,"{'rouge-1': {'f': 0.6071428521444515, 'p': 0.6...",0.75
7,example,1,0.453636,"{'rouge-1': {'f': 0.5607476585728012, 'p': 0.6...",0.75


In [20]:
example='example'
output_path = (f'{DATA_PATH}/output-{example}.txt')
with open(output_path, 'r') as file:
    expected_output = json.load(file)

In [30]:
import json
json.loads(example_1[example_1.prompt_version==6].llm_change_report.values[0])

{'changes': [{'type': 'Query',
   'field': 'getWeather',
   'change': "Parameter name changed from 'location' to 'city'",
   'breaking': True,
   'release_note': 'The parameter name for `getWeather` query has been changed from `location` to `city`. This is a breaking change as existing queries using the `location` parameter will need to be updated.'},
  {'type': 'Weather',
   'field': 'visibility',
   'change': "Added new Int field 'visibility'",
   'breaking': False,
   'release_note': 'A new field `visibility` has been added to the `Weather` type, providing visibility information. This is a non-breaking change.'}],
 'release_notes': {'summary': 'This release contains one breaking change and one enhancement. The breaking change involves renaming the parameter in the `getWeather` query from `location` to `city`. The enhancement adds a new `visibility` field to the `Weather` type. Clients will need to update their queries to use the new parameter name, but can optionally start using the

In [21]:
expected_output

{'changes': [{'type': 'Query',
   'field': 'getWeather',
   'change': "Renamed input parameter 'location' to 'city'",
   'breaking': True,
   'release_note': 'The input parameter for `getWeather` has been renamed from `location` to `city`. This is a breaking change, so make sure to update any queries that use `location` to `city`.'},
  {'type': 'Weather',
   'field': 'visibility',
   'change': "Added new scalar field 'visibility'",
   'breaking': False,
   'release_note': "We've added a new `visibility` field to the `Weather` type. You can now get visibility information in your weather queries without modifying existing ones. This is a non-breaking change."}],
 'release_notes': {'summary': 'This release introduces a breaking change with the renaming of the `location` parameter to `city` in the `getWeather` query, and a non-breaking enhancement with the addition of a new `visibility` field in the `Weather` type.'}}